In [6]:
from BoW import getCountVector
import pandas as pd
from torch import nn
import torch
from ffnn import FFNN
import numpy as np
from scipy.sparse import coo_matrix
from Preprocessing import getVectorLabels
from networkFunctions import finalPrediction, evaluate

In [7]:
df = pd.read_csv("Archive/arxiv_train.csv", nrows=1000)
input, vocabulary = getCountVector(df)
labels = getVectorLabels(df)

In [8]:
model = FFNN(size_vocabulary=input.shape[1])

num_epochs = 30

optimiser = torch.optim.Adam(model.parameters())
loss_function = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    optimiser.zero_grad()
    # Forward pass: Compute predicted y by passing x to the model
    predicted_labels = model(input)

    # Compute and print loss
    loss = loss_function(predicted_labels, labels)
    if epoch % 10 == 0:  # Print every 10 epochs
        print(f'Epoch {epoch}, Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    loss.backward()
    optimiser.step()

Epoch 0, Loss: 2.302584171295166
Epoch 10, Loss: 2.293794631958008
Epoch 20, Loss: 2.2939066886901855


In [ ]:
# Set the model to evaluation mode - important for batch normalization and dropout layers
# Unnecessary in this situation but added for best practices
model.eval()
#size = len(dataloader.dataset)
#num_batches = len(dataloader)
test_loss, correct = 0, 0
test_data = pd.read_csv("Archive/arxiv_test.csv", nrows=100)
count_vectors, _ = getCountVector(test_data, vocabulary)
labels = getVectorLabels(test_data)

# Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
# also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True

with torch.no_grad():

    predictions = model(count_vectors)
    test_loss += loss_function(predictions, labels).item()
    predictions = finalPrediction(predictions)
    accuracy, precision, recall, f1 = evaluate(predictions, labels)

print(f"Test Error: \n Accuracy: {accuracy}, Precision: {precision}, recall: {recall}, f1: {f1}, Avg loss: {test_loss} \n")

Test Error: 
 Accuracy: 0.37, Precision: 0.3846335608835608, recall: 0.4864625080801551, f1: 0.38020536629232277, Avg loss: 2.2680318355560303 



## Gammelt

In [ ]:
# Set the model to evaluation mode - important for batch normalization and dropout layers
# Unnecessary in this situation but added for best practices
model.eval()
#size = len(dataloader.dataset)
#num_batches = len(dataloader)
test_loss, correct = 0, 0
test_data = pd.read_csv("Archive/arxiv_test.csv", nrows=100)
count_vectors = getCountVector(test_data, vocabulary)
labels = getVectorLabels(test_data)

# Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
# also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
with torch.no_grad():
    for i in range(len(count_vectors)):
        count_vector = count_vectors[i]
        label = labels[i]

        pred = model(count_vector)
        test_loss += loss_function(pred, label).item()
        correct += (pred.argmax(1) == label).type(torch.float).sum().item()

test_loss /= 100
correct /= 100
print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

ValueError: Expected input batch_size (100) to match target batch_size (10).